## Usando maker para formar MD e extrair imagens

In [ ]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter(r"C:\Users\fuedj\Documents\Code\RAG_Dr_Voss_v2\drvossv2\data\dr_voss_diary.pdf")
text, _, images = text_from_rendered(rendered) # _ = 'md'

In [ ]:
text

In [ ]:
full_output_path = r"C:\Users\fuedj\Documents\Code\RAG_Dr_Voss_v2\drvossv2\data\dr_voss_md.md"
try:
    with open(full_output_path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"Texto salvo com sucesso em: {full_output_path}")
except IOError as e:
    print(f"Erro ao salvar o arquivo Markdown: {e}")

In [ ]:
import sqlite3

def setup_database(db_name="imagens.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS imagens (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome_arquivo TEXT NOT NULL,
            dados_imagem BLOB NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

setup_database() # Cria o banco de dados e a tabela

In [ ]:
from PIL import Image
import io

def image_to_bytes(image_pil: Image.Image, format: str = "JPEG") -> bytes:
    """Converte um objeto PIL.Image.Image para bytes."""
    byte_arr = io.BytesIO()
    image_pil.save(byte_arr, format=format)
    return byte_arr.getvalue()

# Exemplo de uso:
# Digamos que 'minha_imagem_pil' é um dos valores do seu dicionário 'images'
# imagem_em_bytes = image_to_bytes(minha_imagem_pil, format="JPEG")

def store_images_in_db(images_dict: dict, db_name="imagens.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    for filename, image_pil in images_dict.items():
        # Converter a imagem para bytes (pode escolher PNG ou JPEG)
        image_bytes = image_to_bytes(image_pil, format="JPEG")

        try:
            cursor.execute(
                "INSERT INTO imagens (nome_arquivo, dados_imagem) VALUES (?, ?)",
                (filename, image_bytes)
            )
            print(f"Imagem '{filename}' inserida com sucesso.")
        except sqlite3.Error as e:
            print(f"Erro ao inserir a imagem '{filename}': {e}")

    conn.commit()
    conn.close()

# --- Como você usaria com seu dicionário 'images' ---
# Supondo que 'images' é o dicionário resultante do seu código Marker:
# images = {'_page_0_Picture_1.jpeg': <PIL.Image.Image object>, ...}
store_images_in_db(images)

In [ ]:
images

In [ ]:
import sqlite3
from PIL import Image
import io
import base64
import os

# --- Configuração do Banco de Dados SQLite ---
# Este é o arquivo único do banco de dados que você irá transportar.
DB_FILE = 'pdf_images_single_table.db'

# Conexão com o banco de dados SQLite. Se o arquivo não existir, ele será criado.
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

print(f"Banco de dados SQLite '{DB_FILE}' conectado e/ou criado.")

# --- Criação da Tabela Única ---
# Cada linha representa uma imagem específica de um PDF.
# 'nome_do_pdf' é usado para agrupar as imagens de um mesmo PDF.
cursor.execute('''
CREATE TABLE IF NOT EXISTS pdf_images_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome_do_pdf TEXT NOT NULL,
    nome_da_imagem TEXT NOT NULL,
    image_data_base64 TEXT NOT NULL
    UNIQUE
);
''')
conn.commit()

# --- Simulação dos seus dados de PDFs e Imagens ---
# Reutilizamos a função e os dados simulados do exemplo anterior.
def create_dummy_image(text, size=(100, 50)):
    """Cria uma imagem PIL de exemplo."""
    img = Image.new('RGB', size, color = (73, 109, 137))
    from PIL import ImageDraw, ImageFont
    d = ImageDraw.Draw(img)
    try:
        fnt = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        fnt = ImageFont.load_default() # Fallback
    d.text((10,10), text, font=fnt, fill=(255,255,0))
    return img

# Estrutura de dados simulada, onde cada item representa um 'PDF' com suas 'images_pdf'
mock_pdfs_data = [
    {
        'pdf_filename': 'relatorio_vendas_2024.pdf',
        'images_data': [
            {'_page_0_grafico_1.jpeg': create_dummy_image("Grafico Vendas Q1")},
            {'_page_0_tabela_dados.png': create_dummy_image("Tabela Detalhes Q1")},
            {'_page_1_logo_empresa.png': create_dummy_image("Logo Empresa")}
        ]
    },
    {
        'pdf_filename': 'documentacao_tecnica_v1.pdf',
        'images_data': [
            {'_page_0_diagrama_fluxo.jpeg': create_dummy_image("Diagrama Fluxo")},
            {'_page_1_esquema_circuito.png': create_dummy_image("Esquema Circuito")},
            {'_page_2_foto_prototipo.jpeg': create_dummy_image("Foto Prototipo")}
        ]
    }
]

# --- Função para converter PIL Image para Base64 ---
def image_to_base64(image: Image.Image, img_format: str = "JPEG") -> str:
    """Converte um objeto PIL Image para uma string Base64."""
    buffered = io.BytesIO()
    # Salva a imagem no buffer com o formato especificado.
    image.save(buffered, format=img_format)
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# --- Processamento e Armazenamento na Tabela Única ---
for pdf_info in mock_pdfs_data:
    current_pdf_name = pdf_info['pdf_filename']
    images_pdf_list = pdf_info['images_data'] # Sua lista de dicionários [{nome:img_obj}, ...]

    print(f"\nProcessando dados para o PDF: '{current_pdf_name}'...")

    for image_dict in images_pdf_list:
        for image_name, pil_image_object in image_dict.items():
            try:
                # Tenta inferir o formato da imagem pelo nome do arquivo.
                img_ext = image_name.split('.')[-1].upper() if '.' in image_name else 'JPEG'
                img_format = img_ext if img_ext in ['JPEG', 'PNG', 'GIF', 'BMP'] else 'JPEG'
                if img_format == 'JPG': img_format = 'JPEG' # Normaliza JPG para JPEG

                base64_image_string = image_to_base64(pil_image_object, img_format)

                # Extrai o número da página se o nome da imagem seguir o padrão '_page_X_...'
                page_number = None
                if '_page_' in image_name:
                    try:
                        page_str = image_name.split('_page_')[1].split('_')[0]
                        page_number = int(page_str)
                    except (ValueError, IndexError):
                        pass # Ignora se não conseguir extrair

                # Insere os dados na tabela. Usa INSERT OR REPLACE para atualizar se já existir
                # (útil se você rodar o script várias vezes com os mesmos nomes de PDF/imagem)
                cursor.execute(
                    """
                    INSERT OR REPLACE INTO pdf_images_data
                    (nome_do_pdf, nome_da_imagem, image_data_base64)
                    VALUES (?, ?, ?)
                    """,
                    (current_pdf_name, image_name, base64_image_string)
                )
                print(f"  - Imagem '{image_name}' do PDF '{current_pdf_name}' salva/atualizada.")
            except Exception as e:
                print(f"  - Erro ao processar ou salvar imagem '{image_name}': {e}")
    conn.commit() # Commita todas as inserções/atualizações para este PDF
    conn.close()
    print(f"Todos os dados de imagem para '{current_pdf_name}' processados e salvos.")

print("\nProcessamento de todos os PDFs concluído.")


Banco de dados SQLite 'pdf_images_single_table.db' conectado e/ou criado.

Processando dados para o PDF: 'relatorio_vendas_2024.pdf'...
  - Imagem '_page_0_grafico_1.jpeg' do PDF 'relatorio_vendas_2024.pdf' salva/atualizada.
  - Imagem '_page_0_tabela_dados.png' do PDF 'relatorio_vendas_2024.pdf' salva/atualizada.
  - Imagem '_page_1_logo_empresa.png' do PDF 'relatorio_vendas_2024.pdf' salva/atualizada.
Todos os dados de imagem para 'relatorio_vendas_2024.pdf' processados e salvos.

Processando dados para o PDF: 'documentacao_tecnica_v1.pdf'...
  - Imagem '_page_0_diagrama_fluxo.jpeg' do PDF 'documentacao_tecnica_v1.pdf' salva/atualizada.
  - Imagem '_page_1_esquema_circuito.png' do PDF 'documentacao_tecnica_v1.pdf' salva/atualizada.
  - Imagem '_page_2_foto_prototipo.jpeg' do PDF 'documentacao_tecnica_v1.pdf' salva/atualizada.
Todos os dados de imagem para 'documentacao_tecnica_v1.pdf' processados e salvos.

Processamento de todos os PDFs concluído.


In [9]:
# --- Exemplo de Recuperação e Decodificação de Dados ---
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()
print("\n--- Recuperando e decodificando imagens para um PDF específico ---")

def base64_to_image(base64_string: str) -> Image.Image:
    """Converte uma string Base64 de volta para um objeto PIL Image."""
    img_bytes = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(img_bytes))

target_pdf_to_retrieve = 'relatorio_vendas_2024.pdf'

try:
    # Seleciona todas as imagens associadas ao 'target_pdf_to_retrieve'
    cursor.execute(
        "SELECT nome_da_imagem, image_data_base64 FROM pdf_images_data WHERE nome_do_pdf = ?",
        (target_pdf_to_retrieve,)
    )
    retrieved_images_records = cursor.fetchall()

    if retrieved_images_records:
        print(f"Imagens encontradas para o PDF '{target_pdf_to_retrieve}':")
        # Recria a estrutura que você tinha originalmente: [{nome_img: objeto_pil}, ...]
        reconstructed_images_pdf_list = []
        for img_name, base64_data, img_format in retrieved_images_records:
            try:
                pil_image = base64_to_image(base64_data)
                reconstructed_images_pdf_list.append({img_name: pil_image})
                print(f"  - Imagem '{img_name}' ({img_format}) decodificada e objeto PIL criado.")
            except Exception as e:
                print(f"  - Erro ao decodificar imagem '{img_name}': {e}")

        print(f"\nSua lista 'images_pdf' para '{target_pdf_to_retrieve}' foi reconstruída:")
        for img_dict in reconstructed_images_pdf_list:
            for name, img_obj in img_dict.items():
                print(f"    - Nome: {name}, Objeto PIL: {img_obj}")

        # Você agora tem 'reconstructed_images_pdf_list' com os objetos PIL
        # e pode usá-lo como o 'images_pdf' original.

    else:
        print(f"Nenhuma imagem encontrada para o PDF '{target_pdf_to_retrieve}'.")

except Exception as e:
    print(f"Erro ao recuperar dados para '{target_pdf_to_retrieve}': {e}")

finally:
    # Fechar a conexão com o banco de dados
    conn.close()
    print(f"\nConexão com o banco de dados '{DB_FILE}' fechada.")

# --- Confirmação de Portabilidade ---
print(f"\nO arquivo de banco de dados '{DB_FILE}' agora contém todos os seus dados.")
print(f"O tamanho do arquivo é: {os.path.getsize(DB_FILE) / (1024*1024):.2f} MB")


--- Recuperando e decodificando imagens para um PDF específico ---
Imagens encontradas para o PDF 'relatorio_vendas_2024.pdf':
Erro ao recuperar dados para 'relatorio_vendas_2024.pdf': not enough values to unpack (expected 3, got 2)

Conexão com o banco de dados 'pdf_images_single_table.db' fechada.

O arquivo de banco de dados 'pdf_images_single_table.db' agora contém todos os seus dados.
O tamanho do arquivo é: 0.05 MB
